In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

import os
import yaml
from crewai import Agent, Task, Crew, LLM

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [5]:
from crewai_tools import FileReadTool
csv_tool = FileReadTool(file_path='./support_tickets_data.csv')

In [7]:
llm=LLM(model="ollama/llama3.2", base_url="http://localhost:11434")

In [9]:
# Creating Agents
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  tools=[csv_tool],
    llm=llm
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  tools=[csv_tool],
    llm=llm
)

chart_generation_agent = Agent(
  config=agents_config['chart_generation_agent'],
  allow_code_execution=True,
    llm=llm
)

# Creating Tasks
suggestion_generation = Task(
  config=tasks_config['suggestion_generation'],
  agent=suggestion_generation_agent
)

table_generation = Task(
  config=tasks_config['table_generation'],
  agent=reporting_agent
)

chart_generation = Task(
  config=tasks_config['chart_generation'],
  agent=chart_generation_agent
)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=reporting_agent,
  context=[suggestion_generation, table_generation, chart_generation]
)


# Creating Crew
support_report_crew = Crew(
  agents=[
    suggestion_generation_agent,
    reporting_agent,
    chart_generation_agent
  ],
  tasks=[
    suggestion_generation,
    table_generation,
    chart_generation,
    final_report_assembly
  ],
  verbose=True
)


In [ ]:
support_report_crew.test(n_iterations=1, openai_model_name='ollama/llama3.2')

2024-11-06 20:26:47,770 - 20660 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Using tool: Read a file's content
## Tool Input: 
"{\"name\": \"support_tickets_data.csv\", \"args_schema\": \"None, 'return_direct': False, 'verbose': False, 'callbacks': None, 'callback_manager': None, 'tags': None, 'metadata': None, 'handle_tool_error': None, 'handle_validation_error': None, 'response_format': 'content\"}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_d